# Regression Models in R (tips)

In [1]:
install.packages("foreign")

Installing package into 'C:/Users/ytsat/Documents/R/win-library/4.0'
(as 'lib' is unspecified)



package 'foreign' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'foreign'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\ytsat\Documents\R\win-library\4.0\00LOCK\foreign\libs\x64\foreign.dll to C:\Users\ytsat\Documents\R\win-library\4.0\foreign\libs\x64\foreign.dll: Permission denied"
Warning message:
"restored 'foreign'"



The downloaded binary packages are in
	C:\Users\ytsat\AppData\Local\Temp\RtmpWS5Nl4\downloaded_packages


In [2]:

library(readr)
library(dplyr)
library(ggplot2)
library(car)


Warning message:
"package 'dplyr' was built under R version 4.0.2"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'ggplot2' was built under R version 4.0.2"
Warning message:
"package 'car' was built under R version 4.0.2"
Loading required package: carData


Attaching package: 'car'


The following object is masked from 'package:dplyr':

    recode




In [3]:
############################################################################
#####   Package mechkar                                                 ####
#####   Author: Tomas Karpati M.D.                                      ####
#####   Creation date: 2017-05-01                                       ####
############################################################################

############################################################################
#####   DATA VISUALIZATION                                              ####
#####   Author: Tomas Karpati M.D.                                      ####
#####   Creation date: 2014-03-12                                       ####
############################################################################



############################################################################
#####   TABLE 1                                                         ####
#####   Author: Tomas Karpati M.D.                                      ####
#####   Creation date: 2016-03-09                                       ####
#####   Last Modified: 2018-12-19                                       ####
############################################################################

####################  FUNCTIONS  ###########################################
#### Usage:
####   x: character vector with the name of the variables
####   y: the name of the strata variable (optional)
####   rn: character vector with the text we want to replace the variable names
####   data: the dataset to be used
####   miss: include missing statistics: [0=none, 1=only for categorical variables, 2=for all variables]
####   excel: export the table to excel [0=no, 1=yes]
####   excel_file: the name of the excel file we want to save the table (optional)
####
###################

Table1 <- function(x=NULL, y=NULL, rn=NULL, data=NULL, miss=3, catmiss=TRUE, formatted=TRUE, categorize=FALSE,
                    factorVars=NULL, maxcat=10, delzero=TRUE, decimals=1, messages=TRUE, excel=0, excel_file=NULL,
                    debug=FALSE) {
  ### define sub-functions
  Del <- NULL
  Pop <- NULL
  n <- NULL
  g1 <- function(var)c(Mean=mean(var,na.rm=TRUE), SD=stats::sd(var,na.rm=TRUE))
  g2 <- function(var)c(Median=stats::median(var,na.rm=TRUE), IQR=stats::quantile(var,c(0.25,0.75),na.rm=TRUE))
  msg <- NULL

  ### function for transforming variables to factors
  setFactors <- function(data=data, factorVars=factorVars, catmiss=catmiss, maxcat=maxcat) {
    if(is.null(factorVars)==TRUE) {
      aa <- sapply(sapply(data, unique), length)
      factorVars <- names(which(aa <= maxcat))
    }
    for (v in factorVars) {
      ct <- ifelse( ((is.null(factorVars)==FALSE & (v %in% factorVars)) | (is.null(factorVars)==TRUE & length(unique(data[[v]])) <= maxcat)),1,0)
      if (ct == 1) {
        data[[v]] <- factor(data[[v]])
        if(catmiss == TRUE & sum(is.na(data[[v]])==TRUE) > 0) {
          data[[v]] <- factor(data[[v]],levels=c(levels(data[[v]]),"Missing"))
          data[[v]][which(is.na(data[[v]])==TRUE)] <- "Missing"
        }
      }
    }
    return(data)
  }
  ### proceed to convert varibles to factors
  if (categorize == TRUE | is.null(factorVars)==FALSE ) {
    data <- setFactors(data, factorVars, catmiss, maxcat)
  }

  getSimpleTable  <- function(x=x, rn=rn, data=data, miss=miss, catmiss=catmiss,formatted=formatted,
                              categorize=categorize,maxcat=maxcat, delzero=delzero) {
    if (is.null(x)==TRUE) { x <- names(data)}
    if (is.null(rn)==TRUE) { rn <- x}
    ln <- length(x)
    pb <- utils::txtProgressBar(min=0,max=ln,style=3)
    msg <- NULL
    ### define the column names
    tableaaaa <- cbind(Del="Del",V1="Variables",V2="Categories",n="n","Population")
    tablebbbb <- cbind(Del="Del",V1="Variables",V2="Categories",n="n",val1="val1",val2="val2",val3="val3")
    tbl1 <- cbind(0,"Individuals","n",n=1, nrow(data))
    tbl2 <- cbind(0,"Individuals","n",n=1, nrow(data),NA,NA)
    tableaaaa <- rbind(tableaaaa,tbl1)
    tablebbbb <- rbind(tablebbbb,tbl2)
    q <- 1
    n <- 1
    ii <- 1
    for (v in x)
    {
      if (v %in% names(data)) {
        ### define if the actual variable has to be treated as numeric or factor
        ct <- ifelse(is.numeric(data[[v]])==TRUE & categorize==TRUE &
                       ((is.null(factorVars)==FALSE & (v %in% factorVars)) |
                          (is.null(factorVars)==TRUE & length(unique(data[[v]])) <= maxcat)),1,0)
        ### treat as numeric
        if (length(unique(data[v]))==0) {
          if (messages==TRUE) {
            msg <- c(msg, paste("The variable",v,"has no data... avoided"))
          }
        } else if (inherits(data[[v]], "Date")==TRUE) {
          if (messages==TRUE) {
            msg <- c(msg, paste("The variable",v,"is a date. Dates are not allowed in Table1... avoided"))
          }
        } else if (is.numeric(data[[v]])==TRUE & ct==0) {
          ## report mean and standard deviation
          t_n <- g1(data[[v]])
          tp <- paste(format(round(t_n[1],decimals),nsmall=1,big.mark=",")," (", format(round(t_n[2],decimals),nsmall=1,big.mark=","),")",sep="")
          tbl1 <- cbind(0,rn[q],"Mean (SD)",n=1, tp)
          tbl2 <- cbind(0,rn[q],"Mean (SD)",n=1,t_n[1],t_n[2],NA)
          tableaaaa <- rbind(tableaaaa,tbl1)
          tablebbbb <- rbind(tablebbbb,tbl2)
          ## report median and Interquartile ranges (25%,75%)
          t_n <- g2(data[[v]])
          tp <- paste(format(round(t_n[1],decimals),nsmall=1,big.mark=",")," (", format(round(t_n[2],decimals),nsmall=1,big.mark=","),"-", format(round(t_n[3],decimals),nsmall=1,big.mark=","), ")",sep="")
          tbl1 <- cbind(0,rn[q],"Median (IQR)",n=2, format(tp,big.mark=","))
          tbl2 <- cbind(0,rn[q],"Median (IQR)",n=2,t_n[1],t_n[2],t_n[3])
          tableaaaa <- rbind(tableaaaa,tbl1)
          tablebbbb <- rbind(tablebbbb,tbl2)
          ## report number and percent of missing
          if (miss >= 1) {
            datams <- subset(data,is.na(data[[v]])==TRUE)
            if (nrow(datams)>0) {
              data$cnt <- 1
              datams$cnt <- 1
              t_n <- table(data$cnt)
              t_m <- sum(datams$cnt)
              tp <- paste(format(t_m,big.mark=",")," (",format(round((t_m/t_n)*100,decimals),nsmall=1,big.mark=","),"%)",sep="")
              tbl1 <- cbind(0,rn[q],"Missing (%)",n=3, tp)
              tbl2 <- cbind(0,rn[q],"Missing (%)",n=3, t_m, (t_m/t_n)*100, NA)
            } else {
              tbl1 <- cbind(1,rn[q],"Missing (%)",n=3, " -- ")
              tbl2 <- cbind(1,rn[q],"Missing (%)",n=3, NA, NA, NA)
            }
            tableaaaa <- rbind(tableaaaa,tbl1)
            tablebbbb <- rbind(tablebbbb,tbl2)
          }
        } else {
          t_n <- table(data[[v]])
          ttotal <- sum(t_n)
          nm <- row.names(t_n)
          for (f in 1:length(nm)) {
            del1 <- ifelse(length(nm)==2 & (nm[f]=="No" | nm[f]=="no" | nm[f]==0 | nm[f]=="0" | nm[f]=="None" | nm[f]=="none"),1,0)
            tp <- t_n[f] / ttotal * 100
            pct <- paste(format(round(t_n[f],decimals),nsmall=0,big.mark=",")," (", format(round(tp,decimals),nsmall=1,big.mark=","), "%)",sep="")
            tbl1 <- cbind(del1,rn[q],nm[f],n=f, pct)             ########### delete rows 0/1 !!!!!!!!!
            tbl2 <- cbind(del1,rn[q],nm[f],n=f, t_n[f], tp, NA)  ########### delete rows 0/1 !!!!!!!!!
            tableaaaa <- rbind(tableaaaa,tbl1)
            tablebbbb <- rbind(tablebbbb,tbl2)
          }
          if (miss >= 2 & catmiss==FALSE ) {
            datams <- subset(data,is.na(data[[v]])==TRUE)
            if (nrow(datams)>0) {
              data$cnt <- 1
              datams$cnt <- 1
              t_n <- table(data$cnt)
              t_m <- sum(datams$cnt)
              tp <- paste(format(t_m,big.mark=",")," (",format(round((t_m/t_n)*100,decimals),nsmall=1,big.mark=","),"%)",sep="")
              tbl1 <- cbind(0,rn[q],"Missing (%)",n=f, tp)
              tbl2 <- cbind(0,rn[q],"Missing (%)",n=f, t_m, (t_m/t_n)*100, NA)
            } else {
              tbl1 <- cbind(1,rn[q],"Missing (%)",n=f, " -- ")
              tbl2 <- cbind(1,rn[q],"Missing (%)",n=f, NA, NA, NA)
            }
            tableaaaa <- rbind(tableaaaa,tbl1)
            tablebbbb <- rbind(tablebbbb,tbl2)
          }
        }
      } else {
        if (messages==TRUE) {
          msg <- c(msg, paste("The variable",v,"doesn't exists in the dataset... avoiding"))
        }
      }
      q <- q + 1
      if(debug==FALSE) {
        utils::setTxtProgressBar(pb,ii)
        ii <- ii + 1
      } else {
        message(v)
      }
    }
    if(formatted==TRUE) {
      return(tableaaaa)
    } else {
      return(tablebbbb)
    }
    close(pb)
  }

  pvals <- function(x=x,y=y,rn=rn,data=data,categorize=categorize,maxcat=maxcat) {
    ptab <- NULL
    if (is.null(y)==FALSE) {
      if (y %in% names(data)) {
        if (is.null(x)==TRUE) { x <- names(data)}
        if (is.null(rn)==TRUE | length(rn)<2) {rn <- x}
        q <- 1
        ptab <- cbind(V="Variables",pval="pval", n="n")

        ln <- length(x)
        ii <- 0
        pb <- utils::txtProgressBar(min=0,max=ln,style=3)

        for (v in x) {
          if (v %in% names(data)) {
            ct <- ifelse(is.numeric(data[[v]])==TRUE & categorize==TRUE & length(unique(data[[v]])) <= maxcat,1,0)
            if (is.numeric(data[[y]])==TRUE & categorize==TRUE & length(unique(data[[y]])) <= maxcat) {
              data[[y]] <- as.factor(data[[y]])
            } else if (is.numeric(data[[y]])==TRUE) {
              if (messages==TRUE) {
                msg <- c(msg, paste("The variable",y,"is not a factor. Please convert to factor or change the 'categorize' flag to TRUE."))
              }
              pval <- "Please rerun!!!"
            }
            if (is.numeric(data[[v]])==TRUE & length(unique(data[[v]])) > 1 & ct == 0) {
              ### first check for homoscedasticity
              tryCatch({
                if (stats::bartlett.test(data[[v]], data[[y]])[3] >= 0.05) {
                  pval <- suppressMessages(round(as.numeric(suppressMessages(car::Anova(stats::lm(data[[v]] ~ data[[y]])))[1, 4]), 3))
                } else {
                  pval <- suppressMessages(round(as.numeric(suppressMessages(car::Anova(stats::lm(data[[v]] ~ data[[y]]), white.adjust = TRUE))[1, 3]), 3))
                }
              }, warning = function(w) {
                suppressWarnings(w)
                #ww <- "suppress warnings..."
              }, error = function(e) {
                pval <- "---"
              })
            } else if (length(unique(data[[v]]))==1) {
              pval <- NA
            } else {
              if(length(unique(data[[v]])) < 15) {
                if (min(table(data[[v]],data[[y]])) > 5) {
                  pval <- round(as.numeric(stats::chisq.test(data[[v]],data[[y]])$p.val),3)
                } else {
                  if(min(table(data[[v]],data[[y]]))==0) {
                    #in cases where there are cells with zero, we use Fisher's exact test
                    tryCatch(
                      pval <- round(as.numeric(stats::fisher.test(data[[v]],data[[y]], workspace=1e9)$p.val),3),
                      error = function(e) {msg <- c(msg,paste0("Unable to calcualte the Fisher test for variables ",v," and ",y))})
                  } else {
                    pval <- round(as.numeric(stats::kruskal.test(data[[v]],data[[y]], workspace=1e9)$p.val),3)
                  }
                }
              } else {
                pval <- NA
              }
            }
            ptab <- rbind(ptab,cbind(V=rn[q],pval=pval,n=2))
          }
          if(debug==FALSE) {
            utils::setTxtProgressBar(pb,ii)
            ii <- ii + 1
          }
          q <- q + 1
        }
      }
    }
    return(ptab)
  }
  ####################### Begin analysis
  ##### check for x's witch have one unique values...get them out...
  vv <- NULL
  j <- 0
  jj <- NULL
  for(v in x) {
    if(length(unique(data[[v]])) < 2) {
      vv <- c(vv,v)
      j <- j + 1
      jj <- c(jj,j)
    }
  }
  warning(paste("The following variables have unique values and will not be included in the analysis:",vv))
  x <- setdiff(x, vv)
  if(is.null(rn)==FALSE & length(jj)>0) {
    rn <- rn[-jj]
  }

  ##### if y is null then make a simple table
  tabaaa1 <- getSimpleTable(x=x, rn=rn, data=data, miss=miss, catmiss=catmiss,formatted=formatted,categorize=categorize,maxcat=maxcat, delzero=delzero)
  tabaaa1 <- tibble::as_tibble(tabaaa1)
  ############################  CHANGE TO 5 !!!!!!!!!!!!!!
  if(length(tabaaa1) > 5) {
    names(tabaaa1) <- c("Del","V1","V2","n","Pop","pop2","pop3")
  } else {
    names(tabaaa1) <- c("Del","V1","V2","n","Pop")
  }
  ##### if y has two levels, then make a compound comparison
  if (is.null(y)==FALSE){
    if (y %in% names(data)) {
      if (is.factor(data[[y]])==FALSE) {
        if (length(levels(factor(data[[y]]))) > 8) {
          if (messages==TRUE) {
            message("The dependent variable has more than 8 levels, table too large!")
          }
        } else if(min(table(data[[y]]))==0) {
          message("The dependent variable has one or more levels with no individuals assigned!")
        } else {
          data[[y]] <- factor(data[[y]])
        }
      }
      if (length(levels(data[[y]])) >= 2) {
        for (lv in levels(data[[y]])) {
          dtsub <- subset(data, data[[y]]==lv)
          tab <- getSimpleTable(x=x, rn=rn, data=dtsub, miss=miss, catmiss=catmiss, formatted=formatted,categorize=categorize,maxcat=maxcat, delzero=delzero)
          tab <- data.frame(tab)
          ############################  CHANGE TO 5 !!!!!!!!!!!!!!
          if(length(tab) > 5) {
            names(tab) <- c("Del","V1","V2","n",paste0(lv,"_1"),paste0(lv,"_2"),paste0(lv,"_3"))
          } else {
            names(tab) <- c("Del","V1","V2","n",lv)
          }
          ############################  CHANGE TO 5 !!!!!!!!!!!!!!
          tab[1,5] <- lv
          tabaaa1 <- suppressMessages(dplyr::left_join(tabaaa1, tab))
        }
        # what to do with dichotomous variables? We remove the "Zero" label...
        # clean unnecesary rows
        if (delzero == TRUE) {
          tabaaa1 <- dplyr::filter(tabaaa1,Del==0)
        }
        ### calculate the p-value
        ptab <- data.frame(pvals(x=x,y=y,rn=rn,data=data,categorize=categorize,maxcat=maxcat))
        names(ptab) <- c("V1","pval","n")
        tabaaa1 <- suppressMessages(dplyr::left_join(tabaaa1, ptab))
        tabaaa1 <- dplyr::filter(tabaaa1,Pop != " -- ") #%>%
      }
    }
  }

  tabaaa1 <- dplyr::select(tabaaa1,-n)
  tabaaa1 <- dplyr::select(tabaaa1,-Del)

  ##### Join the tables...
  #Sys.setenv(JAVA_HOME="")
  if (excel==1) {
    #wb <- xlsx::createWorkbook()
    #sheet1 <- xlsx::createSheet(wb, sheetName="Table 1")
    #xlsx::addDataFrame(tabaaa1,sheet1)
    #### save and close the workbook
    #xlsx::saveWorkbook(wb, excel_file)
    writexl::write_xlsx(tabaaa1,excel_file)
    return(tabaaa1)
  } else {
    return(tabaaa1)
  }
}

########################## END Table1 ###############

############################################################################
#####   TEST & TRAIN DATASET GENERATION                                 ####
#####   Author: Tomas Karpati M.D.                                      ####
#####   Creation date: 2016-08-17                                       ####
############################################################################

train_test <- function(data=NULL,train_name=NULL,test_name=NULL,prop=NULL,seed=123,tableone=FALSE)
{
  pval <- NULL
  checkTrainTest <- function(train=NULL,test=NULL) {
    train[["traintest_ind_"]] <- 1
    test[["traintest_ind_"]] <- 2
    df <- rbind(train, test)
    tab <- Table1(data=df, y="traintest_ind_",messages = FALSE)
    vars <- subset(tab, pval < 0.05)$V1
    vars <- setdiff(vars,"traintest_ind_")
    if (length(vars)==0) {
      message(" ")
      message("You got a perfectly balanced training and test datasets")
      message(" ")
    } else {
      message("WARNING: The following variables are not balanced between the training and test datasets:")
      for (v in vars) { message(paste("*",v)) }
      message("You can try to change the seed value until you get a balanced partition.")
      message("Alternatively, you can ommit this warning and exclude those variables from your model")
      message(" ")
    }
    return(tab)
  }
  nm <- 1
  ttenv = as.environment(nm)
  ## set the seed to make your partition reproductible
  set.seed(seed)
  smp_size <- floor(prop * nrow(data))
  train_ind <- sample(seq_len(nrow(data)), size = smp_size)
  assign(train_name, data[train_ind, ], envir=ttenv)
  assign(test_name, data[-train_ind, ], envir=ttenv)
  message(paste("Dataset partitioned into:"))
  message(paste(" + Train dataset:", train_name))
  message(paste(" + Test dataset:", test_name))
  if(tableone==TRUE) {
    tab = checkTrainTest(get(train_name),get(test_name))
    return(tab)
  }
}


######################### END train_test ###############




In [14]:

df <- read.csv("C:\\Users\\ytsat\\Documents\\DataScience\\data\\tips.csv")

In [15]:
head(df)
dim(df)

,total_bill,tip,sex,smoker,day,time,size
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
6,25.29,4.71,Male,No,Sun,Dinner,4


[1] 244   7

# EDA

In [16]:
summary(df)

   total_bill         tip             sex               smoker         
 Min.   : 3.07   Min.   : 1.000   Length:244         Length:244        
 1st Qu.:13.35   1st Qu.: 2.000   Class :character   Class :character  
 Median :17.80   Median : 2.900   Mode  :character   Mode  :character  
 Mean   :19.79   Mean   : 2.998                                        
 3rd Qu.:24.13   3rd Qu.: 3.562                                        
 Max.   :50.81   Max.   :10.000                                        
     day                time                size     
 Length:244         Length:244         Min.   :1.00  
 Class :character   Class :character   1st Qu.:2.00  
 Mode  :character   Mode  :character   Median :2.00  
                                       Mean   :2.57  
                                       3rd Qu.:3.00  
                                       Max.   :6.00  

# DATASET PARTITION

In [18]:
tab1 <- train_test(data=df, train_name="train", test_name="test", prop=0.7, seed=5, tableone=TRUE)
tab1

Dataset partitioned into:

 + Train dataset: train

 + Test dataset: test

Warning message in Table1(data = df, y = "traintest_ind_", messages = FALSE):
"The following variables have unique values and will not be included in the analysis: "


  |=============================================================         |  88%

 

You got a perfectly balanced training and test datasets

 



V1,V2,Pop,1,2,pval
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Individuals,n,244,170,74,NA
total_bill,Mean (SD),19.8 (8.9),19.6 (8.9),20.3 (8.9),NA
total_bill,Median (IQR),17.8 (13.3-24.1),17.8 (13.4-23.2),17.7 (13.4-25.5),0.545
tip,Mean (SD),3.0 (1.4),3.0 (1.4),3.0 (1.3),NA
tip,Median (IQR),2.9 (2.0-3.6),3.0 (2.0-3.6),2.9 (2.0-3.5),0.932
sex,Female,87 (35.7%),63 (37.1%),24 (32.4%),NA
sex,Male,157 (64.3%),107 (62.9%),50 (67.6%),0.584
smoker,Yes,93 (38.1%),65 (38.2%),28 (37.8%),1
day,Fri,19 (7.8%),16 (9.4%),3 (4.1%),NA


In [19]:
tab1 %>% filter(pval < 0.05)

V1,V2,Pop,1,2,pval
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
traintest_ind_,Median (IQR),1.0 (1.0-2.0),NA,NA,0


# MODELS

In [20]:
### The error we will use is the RMSE and RMSLE
rmse <- function(y,y_hat) {
    err <- sqrt(sum((y_hat-y)^2,na.rm=T)/length(y))
    return(err)
}

rmsle <- function(y,y_hat) {
    err <- sqrt(sum((log(y_hat+1)-log(y+1))^2,na.rm=T)/length(y))
    return(err)
}


In [21]:
### Table of resulting errors
### Name, Model, RMSE, RMSLE
err_res <- NULL

## Linear Models

In [22]:
## model with only the original variables
mod1 <- lm(tip ~., data=train)
summary(mod1)


Call:
lm(formula = tip ~ ., data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.9615 -0.4672 -0.0690  0.3670  3.9368 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.74383    0.40027   1.858    0.065 .  
total_bill   0.09734    0.01168   8.331 3.32e-14 ***
sexMale     -0.21030    0.16936  -1.242    0.216    
smokerYes    0.17068    0.17518   0.974    0.331    
daySat      -0.13139    0.35826  -0.367    0.714    
daySun       0.15808    0.36983   0.427    0.670    
dayThur     -0.10149    0.40477  -0.251    0.802    
timeLunch   -0.05436    0.46934  -0.116    0.908    
size         0.18151    0.10640   1.706    0.090 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.024 on 161 degrees of freedom
Multiple R-squared:  0.5034,	Adjusted R-squared:  0.4787 
F-statistic:  20.4 on 8 and 161 DF,  p-value: < 2.2e-16


In [23]:
pred1 <- predict(mod1,newdata=test)
rmse(test$tip,pred1)
rmsle(test$tip,pred1)
err_res <- rbind(err_res, data.frame(Name="Base Linear regression", Model="mod1", 
                                     RMSE=rmse(test$tip,pred1), 
                                     RMSLE=rmsle(test$tip,pred1)))

[1] 1.069905

[1] 0.2520264

In [24]:
## model with all the variables
mod2 <- lm(tip ~., data=train)
summary(mod2)


Call:
lm(formula = tip ~ ., data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.9615 -0.4672 -0.0690  0.3670  3.9368 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.74383    0.40027   1.858    0.065 .  
total_bill   0.09734    0.01168   8.331 3.32e-14 ***
sexMale     -0.21030    0.16936  -1.242    0.216    
smokerYes    0.17068    0.17518   0.974    0.331    
daySat      -0.13139    0.35826  -0.367    0.714    
daySun       0.15808    0.36983   0.427    0.670    
dayThur     -0.10149    0.40477  -0.251    0.802    
timeLunch   -0.05436    0.46934  -0.116    0.908    
size         0.18151    0.10640   1.706    0.090 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.024 on 161 degrees of freedom
Multiple R-squared:  0.5034,	Adjusted R-squared:  0.4787 
F-statistic:  20.4 on 8 and 161 DF,  p-value: < 2.2e-16


In [25]:
pred2 <- predict(mod2,newdata=test)
rmse(test$tip,pred2)
rmsle(test$tip,pred2)
err_res <- rbind(err_res, data.frame(Name="Extended Linear regression", Model="mod2", 
                                     RMSE=rmse(test$tip,pred2), 
                                     RMSLE=rmsle(test$tip,pred2)))

[1] 1.069905

[1] 0.2520264

## Desicion trees

In [26]:
library(tree)
library(rpart)

In [27]:
mod3 <- tree(tip ~., data=train)
mod3

Warning message in tree(tip ~ ., data = train):
"NAs introduced by coercion"


node), split, n, deviance, yval
      * denotes terminal node

 1) root 170 339.700 3.003  
   2) total_bill < 20.47 111  77.120 2.437  
     4) total_bill < 16.315 71  34.920 2.105  
       8) total_bill < 13.2 40  21.540 1.906 *
       9) total_bill > 13.2 31   9.727 2.363 *
     5) total_bill > 16.315 40  20.490 3.027 *
   3) total_bill > 20.47 59 160.200 4.068  
     6) total_bill < 44.23 54  88.870 3.811  
      12) size < 3.5 33  43.300 3.434 *
      13) size > 3.5 21  33.530 4.402  
        26) total_bill < 32.055 14  21.100 3.991 *
        27) total_bill > 32.055 7   5.316 5.226 *
     7) total_bill > 44.23 5  29.200 6.846 *

In [28]:
pred3 <- predict(mod3,newdata=test)
rmse(test$tip,pred3)
rmsle(test$tip,pred3)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-tree", Model="mod3", 
                                     RMSE=rmse(test$tip,pred3), 
                                     RMSLE=rmsle(test$tip,pred3)))

Warning message in pred1.tree(object, tree.matrix(newdata)):
"NAs introduced by coercion"


[1] 1.112837

[1] 0.2550448

In [29]:
mod4 <- rpart(tip ~., data=train)
mod4

n= 170 

node), split, n, deviance, yval
      * denotes terminal node

 1) root 170 339.747800 3.003294  
   2) total_bill< 20.47 111  77.119490 2.437477  
     4) total_bill< 16.315 71  34.918760 2.105493  
       8) total_bill< 13.2 40  21.536380 1.905750  
        16) day=Fri,Sat,Thur 32   4.463800 1.720000 *
        17) day=Sun 8  11.552090 2.648750 *
       9) total_bill>=13.2 31   9.727277 2.363226 *
     5) total_bill>=16.315 40  20.485880 3.026750  
      10) smoker=No 29  13.544990 2.820690 *
      11) smoker=Yes 11   2.463200 3.570000 *
   3) total_bill>=20.47 59 160.235000 4.067797  
     6) total_bill< 40.87 52  85.926900 3.764808  
      12) day=Fri,Sat 20  29.555520 3.182000 *
      13) day=Sun,Thur 32  45.332270 4.129063  
        26) size< 3.5 16  14.336890 3.610625 *
        27) size>=3.5 16  22.394500 4.647500 *
     7) total_bill>=40.87 7  34.072490 6.318571 *

In [30]:
pred4 <- predict(mod4,newdata=test)
rmse(test$tip,pred4)
rmsle(test$tip,pred4)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-rpart", Model="mod4", 
                                     RMSE=rmse(test$tip,pred4), 
                                     RMSLE=rmsle(test$tip,pred4)))

[1] 1.193916

[1] 0.2719324

## Random Forest

In [31]:
library(randomForest)
library(ranger)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


The following object is masked from 'package:dplyr':

    combine



Attaching package: 'ranger'


The following object is masked from 'package:randomForest':

    importance




In [32]:
#mod5 <- randomForest(tip ~., data=train)
#mod5

In [33]:
#pred5 <- predict(mod5,newdata=test)
#rmse(test$tip,pred5)
#rmsle(test$tip,pred5)

In [34]:
mod6 <- ranger(tip ~., data=train)
mod6

Ranger result

Call:
 ranger(tip ~ ., data = train) 

Type:                             Regression 
Number of trees:                  500 
Sample size:                      170 
Number of independent variables:  6 
Mtry:                             2 
Target node size:                 5 
Variable importance mode:         none 
Splitrule:                        variance 
OOB prediction error (MSE):       1.144227 
R squared (OOB):                  0.4308295 

In [35]:
pred6 <- predict(mod6,data=test)
#head(pred6)
rmse(test$tip,pred6$predictions)
rmsle(test$tip,pred6$predictions)
err_res <- rbind(err_res, data.frame(Name="RandomForest (ranger)", Model="mod6", 
                                     RMSE=rmse(test$tip,pred6$predictions), 
                                     RMSLE=rmsle(test$tip,pred6$predictions)))

[1] 1.042714

[1] 0.2500861

## XGBoost

In [36]:
library(xgboost)

Warning message:
"package 'xgboost' was built under R version 4.0.3"

Attaching package: 'xgboost'


The following object is masked from 'package:dplyr':

    slice




In [37]:
train1 <- Matrix::sparse.model.matrix(tip ~ .-1, data = train)

In [38]:
test1 <- Matrix::sparse.model.matrix(tip ~ .-1, data = test)

In [39]:
#X_train <- xgb.DMatrix(train1)
X_train <- train1
y_train <- train$tip
mod7 <- xgboost(data=X_train,label=y_train, nrounds=100,print_every_n = 10)

[1]	train-rmse:2.152302 
[11]	train-rmse:0.428750 
[21]	train-rmse:0.273885 
[31]	train-rmse:0.207797 
[41]	train-rmse:0.149647 
[51]	train-rmse:0.101319 
[61]	train-rmse:0.078424 
[71]	train-rmse:0.056676 
[81]	train-rmse:0.045739 
[91]	train-rmse:0.033742 
[100]	train-rmse:0.027106 


In [40]:
#X_test <- xgb.DMatrix(test1)
X_test <- test1
y_test <- test$tip

pred7 <- predict(mod7,newdata=X_test)
rmse(y_test,pred7)
rmsle(y_test,pred7)
err_res <- rbind(err_res, data.frame(Name="XGBoost", Model="mod7", 
                                     RMSE=rmse(test$tip,pred7), 
                                     RMSLE=rmsle(test$tip,pred7)))

[1] 1.252842

[1] 0.3033884

## kNN 

In [41]:
### adaboost needs that values to be normalized
min_max <- function(x) { (x -min(x))/(max(x)-min(x))   }

In [42]:
X_train <- sapply(data.frame(as.matrix(train1)),min_max)

In [43]:
X_test <- sapply(data.frame(as.matrix(test1)),min_max)

In [44]:
summary(X_train)

   total_bill       sexFemale         sexMale         smokerYes     
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.2159   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :0.3084   Median :0.0000   Median :1.0000   Median :0.0000  
 Mean   :0.3454   Mean   :0.3706   Mean   :0.6294   Mean   :0.3824  
 3rd Qu.:0.4207   3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :1.0000  
     daySat           daySun       dayThur         timeLunch        size       
 Min.   :0.0000   Min.   :0.0   Min.   :0.0000   Min.   :0.0   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:0.0   1st Qu.:0.0000   1st Qu.:0.0   1st Qu.:0.2000  
 Median :0.0000   Median :0.0   Median :0.0000   Median :0.0   Median :0.2000  
 Mean   :0.3412   Mean   :0.3   Mean   :0.2647   Mean   :0.3   Mean   :0.3106  
 3rd Qu.:1.0000   3rd Qu.:1.0   3rd Qu.:1.0000   3rd Qu.:1.0   3rd Qu.:0.4000  
 Max.   :1.0000   Max.   :1.0   Max. 

In [45]:
library(class)
mod8 <- knn(X_train,X_test,cl=train$tip)

In [46]:
str(mod8)

 Factor w/ 89 levels "1","1.1","1.17",..: 49 47 11 40 79 47 16 48 48 33 ...


In [47]:
pred8 <- as.numeric(as.character(mod8))

rmse(test$tip,pred8)
rmsle(test$tip,pred8)
err_res <- rbind(err_res, data.frame(Name="kNN", Model="mod8", 
                                     RMSE=rmse(test$tip,pred8), 
                                     RMSLE=rmsle(test$tip,pred8)))

[1] 1.285067

[1] 0.3250757

## SVM

In [48]:
#install.packages("liquidSVM")
library(liquidSVM)

mod9 <- svm(tip ~., train)

Warning message:
"package 'liquidSVM' was built under R version 4.0.3"
Warning message in (function (model, command.args = NULL, ..., d = NULL, warn.suboptimal = getOption("liquidSVM.warn.suboptimal", :
"Solution may not be optimal: try training again using max_gamma=25"


In [49]:
pred9 <- predict(mod9, newdata=test)

rmse(test$tip,pred9)
rmsle(test$tip,pred9)
err_res <- rbind(err_res, data.frame(Name="SVM", Model="mod9", 
                                     RMSE=rmse(test$tip,pred9), 
                                     RMSLE=rmsle(test$tip,pred9)))

[1] 1.009309

[1] 0.2450502

In [50]:
err_res %>% arrange(RMSLE)

Name,Model,RMSE,RMSLE
<chr>,<chr>,<dbl>,<dbl>
SVM,mod9,1.009309,0.2450502
RandomForest (ranger),mod6,1.042714,0.2500861
Base Linear regression,mod1,1.069905,0.2520264
Extended Linear regression,mod2,1.069905,0.2520264
Decision Trees-tree,mod3,1.112837,0.2550448
Decision Trees-rpart,mod4,1.193916,0.2719324
XGBoost,mod7,1.252842,0.3033884
kNN,mod8,1.285067,0.3250757
